## Autograd: tensor의 모든 연산에 대해 자동 미분 제공
### vector-Jacobian product 사용

requires_grad가 true이면 tensor의 연산에 대해 추적하여 .backward()를 통해 모든 gradient를 얻을 수 있다.

In [1]:
import torch

In [3]:
# requires_grad = False
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

# requires_grad = True
a.requires_grad_(True)
print(a.requires_grad)

# requires_grad=True인 변수를 연산하면 그 변수 또한 true
b = (a * a).sum()
print(b.grad_fn)

False
True


In [4]:
# gradient 계산

x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

out.backward()
print(x.grad) # d(out)/dx

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


/storage/kjwoo31/anaconda3/envs/pytorch/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 803: system has unsupported display driver / cuda driver combination (Triggered internally at  /opt/conda/conda-bld/pytorch_1603729062494/work/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(


In [5]:
# vector의 gradient는 완벽하게 구할 수는 없음. grad를 구하면 vector-Jacobian product 연산을 하게 됨.

x = torch.randn(3, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y) # 위 경우와 달리, backward의 앞에 들어가는 것이 scalar가 아닌 vector

v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([-681.1044, 1054.6636, 1102.9935], grad_fn=<MulBackward0>)
tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


In [7]:
# autorgrad 없애기

# 1. with 사용, 연산하기
print((x).requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)
    
# 2. detach
print(x.requires_grad)

y = x.detach() # x에서 gradient가 필요 없는 y 생성 (내용만 복사)
print(y.requires_grad)
print(x.eq(y).all()) # 내용은 같다

True
True
False
True
False
tensor(True)
